In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline 
%pylab inline
from IPython.core.display import clear_output
import os,shutil
import re
import copy as cp
from scipy import stats
import subprocess
import glob
import pmx
from pmx.AbsoluteDG import *

Populating the interactive namespace from numpy and matplotlib
RDKit imports failed


In [2]:
#!jt -t chesterish

------------------------------
Perform absolute binding free energy calculations
-----------------------------------

1. setup folder structure: water and complex
2. prepare simulation setup
3. run simulations: em, posre, eq, ti

-----------------
Lysozyme
-----------------

In [3]:
# initialize the free energy environment object: it will store the main parameters for the calculations
fe = AbsoluteDG( ligList=['lysozyme_benzene'], 
                      apoCase='lysozyme_apo',bDSSB=True )

# set the workpath
fe.workPath = './lysopath'
# set the path to the molecular dynamics parameter files
fe.mdpPath = './mdppath'
# set the number of replicas (several repetitions of calculation are useful to obtain reliable statistics)
fe.replicas = 3
# provide the path to the structures and topologies
fe.structTopPath = './struct_top'

# finally, let's prepare the overall free energy calculation directory structure
fe.simTypes = ['em_posre','eq_posre','eq','transitions']

fe.prepareFreeEnergyDir( )

# assemble the systems 
fe.assemble_systems()

# box/water/ions
fe.boxWaterIons()

# em_posre
#fe.prepare_simulation( simType='em_posre' )
fe.JOBsimcpu = 4
#fe.JOBmodules = ['shared','owl/intel-mpi-default','cuda91']
#fe.JOBsource = ['/etc/profile.d/modules.sh']
fe.JOBqueue = 'SLURM'
fe.JOBmodules = ['owl/intel-mpi-default','cuda/11.0','hwloc']
fe.JOBgmx = '/usr/local/gromacs/2021/2021.2-impi2017-fftw337-gcc93-cuda11.1/bin/mdrun_threads_AVX_256'
#fe.prepare_jobscripts( simType='em_posre' )

# eq_posre
#fe.prepare_simulation( simType='eq_posre', prevSim='em_posre' )
fe.JOBsimcpu = 8
#fe.prepare_jobscripts( simType='eq_posre' )

# eq
#fe.prepare_simulation( simType='eq', prevSim='eq_posre' )
fe.JOBsimcpu = 20
#fe.prepare_jobscripts( simType='eq' )

# transitions
fe.equilTime = 1080.0 # ps to discard as equilibration
#fe.bGenTiTpr = True
fe.JOBgmx = '/usr/local/gromacs/2021/2021.2-impi2017-fftw337-gcc93-cuda11.1/bin/mdrun_threads_AVX_256'
#fe.prepare_simulation( simType='transitions' )

# jobscript for transitions
fe.JOBgmx = '/usr/local/gromacs/2021/2021.2-impi2017-fftw337-gcc93-cuda11.1/bin/mdrun_threads_AVX_256'
fe.JOBqueue = 'SLURM'
fe.JOBmodules = ['owl/intel-mpi-default','cuda/11.0','hwloc']
fe.JOBsimcpu = 4
#fe.prepare_jobscripts( simType='transitions' )

# analysis
#fe.run_analysis(ligs=['lysozyme_benzene'] )
#fe.analysis_summary()


---------------------
Summary of the setup:
---------------------

   workpath: /home/vgapsys/project/pymacs/github_degrootlab/pmx/src/pmx/abfe_scripts/lysopath
   mdp path: /home/vgapsys/project/pymacs/github_degrootlab/pmx/src/pmx/abfe_scripts/mdppath
   # ligands: 1
   ligands:
        lysozyme_benzene
   apo state: lysozyme_apo

---------------------
Directory structure:
---------------------

/home/vgapsys/project/pymacs/github_degrootlab/pmx/src/pmx/abfe_scripts/lysopath/
|
|--ligX
|--|--dssb
|--|--|--stateA
|--|--|--|--run1/2/3
|--|--|--|--|--/em_posre/eq_posre/eq/transitions
|--|--|--stateB
|--|--|--|--run1/2/3
|--|--|--|--|--/em_posre/eq_posre/eq/transitions
|--|--strTopFolder
|--lig..

DONE
----------------------
Assembling the systems
----------------------
Order DSSB: ligand-protein-ligand-other-water
--- Assembling structures: lysozyme_benzene ---
--- Assembling topologies: lysozyme_benzene ---
----------------
Box, water, ions
----------------
